# Project 1: Text Classification using BOW and Logistic Regression

For this project, imagine you are a group of data scientists who want to train a neural network to predict the sentiment of a movie review. You are given the set of IMDb reviews you used for Project 1, split to train (*train.jsonl*) and test data (*test.jsonl*).


For this project, you might need the following python packages:
- sklearn
- pandas
- PyTorch

For PyTorch installation, please refer to [PyTorch](https://pytorch.org/get-started/locally/).

### Enter names and mat. numbers:
- Group Name

- name 1, mat 1
- name 2, mat 2
- name 3, mat 3
- name 4, mat 4

In [14]:
# all the imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


## Task 1: Numerical Representation of Texts
As presented in the lecture, we want to represent texts by a "bag of words". Therefore, the text is represented by the bag (or multiset) of its words.

a) Proceed as follows:
1. remove all stop words
2. remove punctuations
3. lower words
2. create the term-document matrix

Hint: You may use your code from Project 1 here.

In [15]:
# your code here

b) write a data class for the BOW dataset and the labels. Transform the categorical labels to numerical labels.

In [16]:
class BOW_data(Dataset):
    # write your code for the dataset implementation here
    def __init__(self, data_points: torch.FloatTensor, class_labels: torch.LongTensor):
        pass
    def __len__(self):
        pass

    def __getitem__(self, index):
        pass

c) Instantiate the train and test data objects.

In [17]:
# your code here

## Task 2: Design the Network
a) Design a neural network with 4 fully connected layers for the task of classifying handwritten digits.
- Use the ReLu activation function after the first three layers.
- Use one of the weight initializers from the lecture to initialize the network's weights.

In [24]:

class Classifier(nn.Module):
    # Write your code for the architecture here...
    def __init__(self, input_size: int, hidden_1_size: int, hidden_2_size: int, hidden_3_size: int, output_size: int):
        pass

    def forward(self, x):
        pass

b) write a function to train the model on batches for a given number of epochs.

In [25]:
def train(clf, train_data, batch_size, epochs, learning_rate=0.0001):
    pass

c) Instantiate the neural network classifier.
- Given the term-document-matrix, what is your input size?
- For the later tasks, try different hidden layer sizes and compare the results.

In [26]:
# your code here

d) Train the model on a batch size of 8 and 3 epochs.
If you run out of memory, further reduce the batch size and the hidden layers' sizes.

In [27]:
# your code here

## Task 3: Evaluate the Neural Network

a) write a function that returns the accuracy of your trained model.

In [28]:
def evaluate(clf, test_data):
    # your code here
    pass

b) Evaluate the model.
- Test at least three different sets parameters for the neural network (hidden sizes)
- Use higher values for the number of training epochs. What changes do you expect?

In [23]:
# your code here